In [1]:
# download pdf results from Tokyo Olympics
# translate pdf reults in pandas df
# graph using altair or some interactive plotting library
# translate this to streamlit or something?

![Image source: https://www.infobae.com/aroundtherings/olympics-games/tokyo-2020/2021/08/04/lasha-talakhadze-smashes-world-and-olympic-records-for-weightlifting-gold/](./lasha.jpeg)
*Lasha Lifting Image source from [Infobae](https://www.infobae.com/aroundtherings/olympics-games/tokyo-2020/2021/08/04/lasha-talakhadze-smashes-world-and-olympic-records-for-weightlifting-gold/)*

# How Good is Lasha?

After his performance in the Tokyo 2020 Olympics, Lasha is definetly in his class of his own.

To be able to be the last lifter following yourself, not to mention this is the olympics. This is something incredible.

But how good is he graphically? Let's see if you can display this graphically based on all the results from the Tokyo 2020 Olympics.

In [2]:
# import libraries
import json
import math
from pathlib import Path
import requests

#import PyPDF2 (does not work)
import tabula #!conda install tabula-py
import pandas as pd
import altair as alt

## Obtaining the pds files

Obtaining the .pdf files using the API for this page. https://olympics.com/tokyo-2020/olympic-games/en/results/weightlifting/reports.htm

In [3]:
wl_links = 'https://olympics.com/tokyo-2020/olympic-games/en/results/weightlifting/zzjp000b.json'

wl_res = requests.get(wl_links)
wl_json = json.loads(wl_res.text)

URLPREFIX = 'https://olympics.com/tokyo-2020/olympic-games'

links = [(link['pdfEvent'], link['pdfLink'].replace('../../..', URLPREFIX)) for link in wl_json['pdfsJSON'] 
         if link['pdfType'] == 'Phase Reports'] # returns a tuple (event name, pdf link)

In [4]:
links[0:3]

[("Men's +109kg",
  'https://olympics.com/tokyo-2020/olympic-games/resOG2020-/pdf/OG2020-/WLF/OG2020-_WLF_C73_WLFMO109KG------------FNL---------.pdf'),
 ("Men's 109kg",
  'https://olympics.com/tokyo-2020/olympic-games/resOG2020-/pdf/OG2020-/WLF/OG2020-_WLF_C73_WLFM109KG-------------FNL---------.pdf'),
 ("Women's +87kg",
  'https://olympics.com/tokyo-2020/olympic-games/resOG2020-/pdf/OG2020-/WLF/OG2020-_WLF_C73_WLFWO87KG-------------FNL---------.pdf')]

In [5]:
# downloading pdfs

for i, tup in enumerate(links):
    test_res = requests.get(links[i][1], stream=True)
    results_path = Path('results/')
    results_path.mkdir(parents=True, exist_ok=True)
    filename = links[i][0] + '.pdf'
    filepath = results_path / filename
    with filepath.open('wb') as f:
        f.write(test_res.content)

## Looking at the results

Let's have a look a the results.

The aim is to plot the [z-scores](https://en.wikipedia.org/wiki/Standard_score) within each category and then compare this among other caretogies.

$ z = \frac{ x - \mu }{ \sigma } $

The z-score is difference between the value and the mean divided by the standard deviation. Mean ($\mu$) is an average; standard deviation ($\sigma$) provides the spread in data.

A very high z-score means the result is exceeds the compeition greatly.

We'll start with just the 109+ weight category.

I've decided not to use [sinclair coefficient](https://en.wikipedia.org/wiki/Sinclair_coefficient) because this is good for a weight to weight comparison. I want to show how a competitor did within his or her weight category.

*TODO: Potentially, save a list of the no totals to display later?*

In [6]:
#PyPDF2 does not work, use tabula

In [7]:
#list all files in results

p = Path('./results').glob('./*.pdf')
files = [_.name.replace('.pdf', '') for _ in p if _.is_file()]
files

for i, j in enumerate(files):
    print(f'{i}: {j}')

0: Women's 76kg
1: Men's 73kg
2: Men's 109kg
3: Women's 55kg
4: Women's 59kg
5: Men's 81kg
6: Women's 49kg
7: Women's 64kg
8: Men's 96kg
9: Women's +87kg
10: Men's +109kg
11: Women's 87kg
12: Men's 61kg
13: Men's 67kg


In [8]:
# Looking into the 109+ class

idx = 10 # 109+ class

result = tabula.read_pdf('results/' + files[idx] + '.pdf', pages=1)
df = result[0]
df['Category'] = files[idx]
df.rename(columns={
                    'NOC\rCode': 'Country',
                    'Unnamed: 5': 'Final Total'
                  },
          inplace=True)
df = df[['Name', 'Rank', 'Country', 'Date of Birth', 'Final Total', 'Category']]
df = df.dropna() # removes NA and those who did not total.

# calculates normalized total within category
df = df.astype({
    'Final Total': 'int'
})
df['Normalised'] = (df['Final Total'] - df['Final Total'].mean()) / df['Final Total'].std() 

print(files[idx])
df

Men's +109kg


,Name,Rank,Country,Date of Birth,Final Total,Category,Normalised
1,TALAKHADZE Lasha,1.0,GEO,2 OCT 1993,488,Men's +109kg,2.513915
2,DAVOUDI Ali,2.0,IRI,22 MAR 1999,441,Men's +109kg,1.013547
3,ASAAD Man,3.0,SYR,20 NOV 1993,424,Men's +109kg,0.470860
4,TOYCHYYEV Hojamuhammet,4.0,TKM,16 JAN 1992,414,Men's +109kg,0.151633
5,LITI David Andrew,5.0,NZL,11 JUL 1996,414,Men's +109kg,0.151633
6,KUWORGE Enzo Kofi,6.0,NED,31 AUG 2001,409,Men's +109kg,-0.007981
7,NAGY Peter,7.0,HUN,16 JAN 1986,396,Men's +109kg,-0.422976
8,RUIZ I VELASCO Marcos,8.0,ESP,13 OCT 1996,395,Men's +109kg,-0.454899
9,WILKES Caine Morgan,9.0,USA,10 JUL 1987,390,Men's +109kg,-0.614513
10,MARTIROSJAN Sargis,10.0,AUT,14 SEP 1986,381,Men's +109kg,-0.901817


In [9]:
# Plot z-scores for 109+ class

alt.Chart(df, title="Distribution of Men's 109+ results",).mark_circle().encode(
    y=alt.Y(
        'jitter:Q',
        title=None,
        axis=alt.Axis(values=[0], ticks=True, grid=False, labels=False),
        scale=alt.Scale(),
        ),
    x=alt.X('Normalised:Q', title='Z-score'),
    color=alt.Color('Rank:Q', legend=None, scale=alt.Scale(scheme='turbo')),
    tooltip=['Name:N', 'Rank:Q', 'Category:N', 'Final Total:Q'],
    ).transform_calculate(
        jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
    ).configure_facet(
        spacing=0
    ).configure_view(
        stroke=None
    )

alt.Chart(...)

## Extrating data from all files

We can see that Lasha is above 2.5 standard deviation from the mean. This is truly amazing. Remember the statistics of standard deviation is not derived from the normal population; this is from the population of olympian he was competing against.

Unfortuntately, the .pdf files don't extra the same. This mean the .pdf files need to be preprocessed differently.


In [33]:
df_all_v1 = []
idx = [0, 8, 10, 11]

for i, file in enumerate(files):
    if i in idx:
        results = tabula.read_pdf('results/' + file + '.pdf', pages='all')
        df = results[0]
        df['Category'] = file
        df.rename(columns={
                            'NOC\rCode': 'Country',
                            'Unnamed: 5': 'Final Total'
                          },
                  inplace=True)
        df = df[['Name', 'Rank', 'Country', 'Date of Birth', 'Final Total', 'Category']]
        df = df.dropna() # removes NA and those who did not total.
        df = df.astype({'Final Total': 'int'})
        df['Normalised'] = (df['Final Total'] - df['Final Total'].mean()) / df['Final Total'].std() 
        df_all_v1.append(df)
df_all_v1 = pd.concat(df_all_v1)
df_all_v1.reset_index(inplace=True)
df_all_v1.drop(columns=['index'], inplace=True)
df_all_v1.head()

,Name,Rank,Country,Date of Birth,Final Total,Category,Normalised
0,DAJOMES BARRERA Neisi Patricia,1.0,ECU,12 MAY 1998,263,Women's 76kg,1.812496
1,NYE Katherine Elizabeth,2.0,USA,5 JAN 1999,249,Women's 76kg,1.074072
2,FUENTES ZAVALA Aremi,3.0,MEX,23 MAY 1993,245,Women's 76kg,0.863093
3,STRENIUS Patricia Caroline,4.0,SWE,23 NOV 1989,235,Women's 76kg,0.335647
4,NAUMAVA Darya,5.0,BLR,26 AUG 1995,234,Women's 76kg,0.282903


In [43]:
df_all_v2 = []
leftover = list(range(len(files)))
leftover = [_ for _ in leftover if _ not in idx] # .pdf files that need to be edited

for i, file in enumerate(files):
    if i in leftover:
        result = tabula.read_pdf('results/' + file + '.pdf', pages=1)
        df = result[0]
        df['Category'] = file
        df['Name'] = df['Lot'].str.replace('\d+', '')
        df.drop(columns=['Unnamed: 1'], inplace=True)
        df.rename(columns={
                            'NOC': 'Country',
                            'Unnamed: 2': 'Date of Birth',
                            'Unnamed: 6': 'Final Total',
                            'Unnamed: 0': 'Rank'
                          },
                  inplace=True)
        df = df[['Name', 'Rank', 'Country', 'Date of Birth', 'Final Total', 'Category']]
        df = df.dropna() # removes those who did not total
        df = df.astype({'Final Total': 'int'})
        df['Normalised'] = (df['Final Total'] - df['Final Total'].mean()) / df['Final Total'].std() 
        df_all_v2.append(df)
df_all_v2 = pd.concat(df_all_v2)
df_all_v2.reset_index(inplace=True)
df_all_v2.drop(columns=['index'], inplace=True)
df_all_v2.head()

,Name,Rank,Country,Date of Birth,Final Total,Category,Normalised
0,SHI Zhiyong,1,CHN,10 OCT 1993,364,Men's 73kg,1.636262
1,MAYORA PERNIA Julio Ruben,2,VEN,9 FEB 1996,346,Men's 73kg,0.841776
2,ABDULLAH Rahmat Erwin,3,INA,13 OCT 2000,342,Men's 73kg,0.665224
3,CALJA Briken,4,ALB,19 FEB 1990,341,Men's 73kg,0.621086
4,ANDREEV Bozhidar Dimitrov,5,BUL,17 JAN 1997,338,Men's 73kg,0.488672


In [54]:
df_all = pd.concat([df_all_v1, df_all_v2])
df_all.info()
df_all.reset_index(inplace=True)
df_all.drop(columns=['index'], inplace=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 175 entries, 0 to 125
Data columns (total 7 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Name           175 non-null    object 
 1   Rank           175 non-null    object 
 2   Country        175 non-null    object 
 3   Date of Birth  175 non-null    object 
 4   Final Total    175 non-null    int64  
 5   Category       175 non-null    object 
 6   Normalised     175 non-null    float64
dtypes: float64(1), int64(1), object(5)
memory usage: 10.9+ KB


In [55]:
df_all.head()

,Name,Rank,Country,Date of Birth,Final Total,Category,Normalised
0,DAJOMES BARRERA Neisi Patricia,1,ECU,12 MAY 1998,263,Women's 76kg,1.812496
1,NYE Katherine Elizabeth,2,USA,5 JAN 1999,249,Women's 76kg,1.074072
2,FUENTES ZAVALA Aremi,3,MEX,23 MAY 1993,245,Women's 76kg,0.863093
3,STRENIUS Patricia Caroline,4,SWE,23 NOV 1989,235,Women's 76kg,0.335647
4,NAUMAVA Darya,5,BLR,26 AUG 1995,234,Women's 76kg,0.282903


In [56]:
df_all.tail()

,Name,Rank,Country,Date of Birth,Final Total,Category,Normalised
170,CHKHEIDZE Goga,8,GEO,11 FEB 1996,302,Men's 67kg,-0.181349
171,DENI .,9,INA,26 JUL 1989,301,Men's 67kg,-0.223199
172,MUNOZ MARTINEZ Jonathan Antonio,10,MEX,10 NOV 1995,298,Men's 67kg,-0.348748
173,ANDRIANTSITOHAINA Tojonirina Alain,11,MAD,26 FEB 1995,285,Men's 67kg,-0.892796
174,KATOATAU Ruben,12,KIR,9 FEB 1997,245,Men's 67kg,-2.566788


In [57]:
df_all.isna().sum() # checking for NaNs

Name             0
Rank             0
Country          0
Date of Birth    0
Final Total      0
Category         0
Normalised       0
dtype: int64

In [58]:
# Plot z-scores for all lifters

alt.Chart(df_all, title="Z-scores for all lifters",).mark_circle().encode(
    y=alt.Y(
        'jitter:Q',
        title=None,
        axis=alt.Axis(values=[0], ticks=True, grid=False, labels=False),
        scale=alt.Scale(),
        ),
    x=alt.X('Normalised:Q', title='Z-score'),
    color=alt.Color('Rank:Q', legend=None, scale=alt.Scale(scheme='turbo')),
    tooltip=['Name:N', 'Rank:Q', 'Category:N', 'Final Total:Q'],
    ).transform_calculate(
        jitter='sqrt(-2*log(random()))*cos(2*PI*random())'
    ).configure_facet(
        spacing=0
    ).configure_view(
        stroke=None
    )

alt.Chart(...)